In [30]:
site_id ="CNU0871"
address = '1515 19th Avenue, San Francisco, California 94122'

In [31]:
from io import StringIO
from docx.shared import Cm
from docxtpl import DocxTemplate, InlineImage
import pandas as pd
from math import log10
import requests


In [32]:
def qb_dataframe(table_id):
    # Get fields
    url = "https://api.quickbase.com/v1/fields?tableId=" + table_id
    payload = {}
    headers = {
      'QB-Realm-Hostname': 'soteriarf',
      'Authorization': 'QB-USER-TOKEN b4pp4v_paqb_eyxwg9bec3v8pb63hjwjb9t36ii',
      'Content': 'application.json'
    }
    response = requests.request("GET", url, headers=headers, data = payload)
    df = pd.json_normalize(response.json())
    df = df[["label", "id", "fieldType", "mode"]]
    df = df.sort_values('id').reset_index(drop=True)
    df = df[df['fieldType'] != 'user']
    df = df[df['fieldType'] != 'file']
    label_list = list(df['label'])
    id_list = list(df['id'])
    #print(label_list, id_list)
    
    # Query the table
    headers = {
      'QB-Realm-Hostname': 'soteriarf.quickbase.com',
      'Authorization': 'QB-USER-TOKEN b4pp4v_paqb_eyxwg9bec3v8pb63hjwjb9t36ii',
      'Content-Type': 'application/json',
    }
    body = {
        "from": table_id,
        "select": id_list,
        "options": {
            "skip": 0,
            "top": 10000,
            #"compareWithAppLocalTime": false
        }} #, "where": "{6.EQ.'" + site_id + "'}"}
    r = requests.post(
        'https://api.quickbase.com/v1/records/query', 
        headers = headers, 
        json = body
    )
    df = pd.json_normalize(r.json()['data'])
    df.columns = df.columns.str.replace(".value", "")
    
    # Rename the dataframe with the correct fields names
    id_list = [str(i) for i in id_list] 
    for col in df.columns:
        df.rename(columns={col:label_list[id_list.index((col))]}, inplace=True)
    return df

In [33]:
# Function to convert from W to dBm
def W2dBm(W):
    return 10*log10(W*1000)

# Function to convert from dBm to mW
def dBm2W(dBm):
    return 10**((dBm)/10) / 1000

In [34]:
W2dBm(1862)

62.69979676645324

In [35]:
gain_dbd = 15.7

In [36]:
dBm2W(W2dBm(661) - 12.4)

38.03657985798617

In [37]:
# Load Transmitters Table
df = qb_dataframe('bqzaae4x9')

# Query on the site
df = df[df['Antenna Location - SITE_ID'] == site_id]
df.head()

,Date Created,Input Power (W),Date Modified,Pattern Code,Pattern Code - Aperture (Feet),Pattern Code - Gain (dBd),Pattern Code - Horizontal BW,Pattern Code - Vertical BW,Record ID#,Pattern Code - Manufacturer,...,Antenna Location - y_roofview,Antenna Location - SITE_ID - Rooftop AGL,Antenna Location - SITE_ID - Roofview AGL,Antenna Location - ANTENNA_ID,Antenna Location - SECTOR,Loss (dB),Antenna Location - RAD_CENTER_FT,Technology,Port,Frequency MHz
36,2020-12-17T03:47:53Z,35.51,2020-12-17T03:47:53Z,NNHH-65A-R4_700,4.6,11.25,71.0,16.6,372,Commscope,...,109.814132,58.5,58.5,CNU0871_ATT_A_1,A,0.0,60,LTE,P01,700
37,2020-12-17T03:47:53Z,142.91,2020-12-17T03:47:53Z,NNHH-65A-R4_850,4.6,11.75,66.0,14.7,373,Commscope,...,109.814132,58.5,58.5,CNU0871_ATT_A_1,A,0.0,60,LTE,P02,850
38,2020-12-17T03:47:53Z,141.93,2020-12-17T03:47:53Z,NNHH-65A-R4_2100,4.6,15.65,66.0,6.5,374,Commscope,...,109.814132,58.5,58.5,CNU0871_ATT_A_1,A,0.0,60,LTE,P03,2100
39,2020-12-17T03:47:53Z,141.95,2020-12-17T03:47:53Z,NNHH-65A-R4_700,4.6,11.25,71.0,16.6,375,Commscope,...,109.876347,58.5,58.5,CNU0871_ATT_A_2,A,0.0,60,LTE,P01,700
40,2020-12-17T03:47:53Z,89.11,2020-12-17T03:47:53Z,NNHH-65A-R4_2300,4.6,15.95,66.0,5.9,376,Commscope,...,109.876347,58.5,58.5,CNU0871_ATT_A_2,A,0.0,60,LTE,P02,2300


In [38]:
list(df)

['Date Created',
 'Input Power (W)',
 'Date Modified',
 'Pattern Code',
 'Pattern Code - Aperture (Feet)',
 'Pattern Code - Gain (dBd)',
 'Pattern Code - Horizontal BW',
 'Pattern Code - Vertical BW',
 'Record ID#',
 'Pattern Code - Manufacturer',
 'z_roofview',
 'sort_key',
 'Related Antenna Location',
 'Antenna Location - SITE_ID',
 'Antenna Location - ANTENNA_MODEL',
 'Antenna Location - AZIMUTH',
 'Antenna Location - SITE_ID - Scale',
 'Antenna Location - x_roofview',
 'Antenna Location - y_roofview',
 'Antenna Location - SITE_ID - Rooftop AGL',
 'Antenna Location - SITE_ID - Roofview AGL',
 'Antenna Location - ANTENNA_ID',
 'Antenna Location - SECTOR',
 'Loss (dB)',
 'Antenna Location - RAD_CENTER_FT',
 'Technology',
 'Port',
 'Frequency MHz']

In [39]:
df = df.rename(columns={
    'Antenna Location - SECTOR': 'sector',
    'Port': 'port',
    'Technology': 'technology',
    'Frequency MHz': 'freq',
    'Pattern Code - Manufacturer': 'manufacturer',
    'Antenna Location - ANTENNA_MODEL': 'model',
    'Antenna Location - AZIMUTH': 'azimuth',
    'Antenna Location - RAD_CENTER_FT': 'agl',
    'Input Power (W)': 'power',
    'Pattern Code - Gain (dBd)': 'gain'
    })
df.head()

,Date Created,power,Date Modified,Pattern Code,Pattern Code - Aperture (Feet),gain,Pattern Code - Horizontal BW,Pattern Code - Vertical BW,Record ID#,manufacturer,...,Antenna Location - y_roofview,Antenna Location - SITE_ID - Rooftop AGL,Antenna Location - SITE_ID - Roofview AGL,Antenna Location - ANTENNA_ID,sector,Loss (dB),agl,technology,port,freq
36,2020-12-17T03:47:53Z,35.51,2020-12-17T03:47:53Z,NNHH-65A-R4_700,4.6,11.25,71.0,16.6,372,Commscope,...,109.814132,58.5,58.5,CNU0871_ATT_A_1,A,0.0,60,LTE,P01,700
37,2020-12-17T03:47:53Z,142.91,2020-12-17T03:47:53Z,NNHH-65A-R4_850,4.6,11.75,66.0,14.7,373,Commscope,...,109.814132,58.5,58.5,CNU0871_ATT_A_1,A,0.0,60,LTE,P02,850
38,2020-12-17T03:47:53Z,141.93,2020-12-17T03:47:53Z,NNHH-65A-R4_2100,4.6,15.65,66.0,6.5,374,Commscope,...,109.814132,58.5,58.5,CNU0871_ATT_A_1,A,0.0,60,LTE,P03,2100
39,2020-12-17T03:47:53Z,141.95,2020-12-17T03:47:53Z,NNHH-65A-R4_700,4.6,11.25,71.0,16.6,375,Commscope,...,109.876347,58.5,58.5,CNU0871_ATT_A_2,A,0.0,60,LTE,P01,700
40,2020-12-17T03:47:53Z,89.11,2020-12-17T03:47:53Z,NNHH-65A-R4_2300,4.6,15.95,66.0,5.9,376,Commscope,...,109.876347,58.5,58.5,CNU0871_ATT_A_2,A,0.0,60,LTE,P02,2300


In [40]:
df = df[['Related Antenna Location', 'sector', 'port', 'technology', 'freq', 'manufacturer', 'model', 'azimuth', 'agl', 'power', 'gain']]
df.head()

,Related Antenna Location,sector,port,technology,freq,manufacturer,model,azimuth,agl,power,gain
36,CNU0871_ATT_A_1,A,P01,LTE,700,Commscope,NNHH-65A-R4,20.0,60,35.51,11.25
37,CNU0871_ATT_A_1,A,P02,LTE,850,Commscope,NNHH-65A-R4,20.0,60,142.91,11.75
38,CNU0871_ATT_A_1,A,P03,LTE,2100,Commscope,NNHH-65A-R4,20.0,60,141.93,15.65
39,CNU0871_ATT_A_2,A,P01,LTE,700,Commscope,NNHH-65A-R4,20.0,60,141.95,11.25
40,CNU0871_ATT_A_2,A,P02,LTE,2300,Commscope,NNHH-65A-R4,20.0,60,89.11,15.95


In [41]:
df['erp_dbm'] = ''
df['erp'] = ''
for x in range(len(df)):
    df['erp_dbm'].iloc[x] = round(W2dBm(df['power'].iloc[x]*1000) + df['gain'].iloc[x], 1)
    df['erp'].iloc[x] = int(dBm2W(df['erp_dbm'].iloc[x]) / 1000)
df = df.drop('erp_dbm', 1)
df['freq'] = df['freq'].astype(int)
df['azimuth'] = df['azimuth'].astype(int)
df['power'] = df['power'].astype(int)
df = df.round({'gain': 1})
df.sort_values(['Related Antenna Location', 'port', 'freq'], ascending=[True, True, True], inplace=True)
df.head()

,Related Antenna Location,sector,port,technology,freq,manufacturer,model,azimuth,agl,power,gain,erp
36,CNU0871_ATT_A_1,A,P01,LTE,700,Commscope,NNHH-65A-R4,20,60,35,11.2,478
37,CNU0871_ATT_A_1,A,P02,LTE,850,Commscope,NNHH-65A-R4,20,60,142,11.8,2137
38,CNU0871_ATT_A_1,A,P03,LTE,2100,Commscope,NNHH-65A-R4,20,60,141,15.6,5248
39,CNU0871_ATT_A_2,A,P01,LTE,700,Commscope,NNHH-65A-R4,20,60,141,11.2,1905
40,CNU0871_ATT_A_2,A,P02,LTE,2300,Commscope,NNHH-65A-R4,20,60,89,16.0,3467


In [42]:
df.iloc[0].to_json(r'json_out.json')
# {"sector":"A","technology":"LTE","freq":2500.0,"manufacturer":"Ericsson","model":"AIR6449","azimuth":10.0,"agl":"96","power":280.0,"gain":22.65,"erp_dbm":77.1,"erp":51286}

In [43]:
row_contents = []
for x in range(len(df)):
    print(df.iloc[x].to_json())
    row_contents.append(eval(df.iloc[x].to_json()))
row_contents

{"Related Antenna Location":"CNU0871_ATT_A_1","sector":"A","port":"P01","technology":"LTE","freq":700,"manufacturer":"Commscope","model":"NNHH-65A-R4","azimuth":20,"agl":"60","power":35,"gain":11.2,"erp":478}
{"Related Antenna Location":"CNU0871_ATT_A_1","sector":"A","port":"P02","technology":"LTE","freq":850,"manufacturer":"Commscope","model":"NNHH-65A-R4","azimuth":20,"agl":"60","power":142,"gain":11.8,"erp":2137}
{"Related Antenna Location":"CNU0871_ATT_A_1","sector":"A","port":"P03","technology":"LTE","freq":2100,"manufacturer":"Commscope","model":"NNHH-65A-R4","azimuth":20,"agl":"60","power":141,"gain":15.6,"erp":5248}
{"Related Antenna Location":"CNU0871_ATT_A_2","sector":"A","port":"P01","technology":"LTE","freq":700,"manufacturer":"Commscope","model":"NNHH-65A-R4","azimuth":20,"agl":"60","power":141,"gain":11.2,"erp":1905}
{"Related Antenna Location":"CNU0871_ATT_A_2","sector":"A","port":"P02","technology":"LTE","freq":2300,"manufacturer":"Commscope","model":"NNHH-65A-R4","azim

[{'Related Antenna Location': 'CNU0871_ATT_A_1',
  'sector': 'A',
  'port': 'P01',
  'technology': 'LTE',
  'freq': 700,
  'manufacturer': 'Commscope',
  'model': 'NNHH-65A-R4',
  'azimuth': 20,
  'agl': '60',
  'power': 35,
  'gain': 11.2,
  'erp': 478},
 {'Related Antenna Location': 'CNU0871_ATT_A_1',
  'sector': 'A',
  'port': 'P02',
  'technology': 'LTE',
  'freq': 850,
  'manufacturer': 'Commscope',
  'model': 'NNHH-65A-R4',
  'azimuth': 20,
  'agl': '60',
  'power': 142,
  'gain': 11.8,
  'erp': 2137},
 {'Related Antenna Location': 'CNU0871_ATT_A_1',
  'sector': 'A',
  'port': 'P03',
  'technology': 'LTE',
  'freq': 2100,
  'manufacturer': 'Commscope',
  'model': 'NNHH-65A-R4',
  'azimuth': 20,
  'agl': '60',
  'power': 141,
  'gain': 15.6,
  'erp': 5248},
 {'Related Antenna Location': 'CNU0871_ATT_A_2',
  'sector': 'A',
  'port': 'P01',
  'technology': 'LTE',
  'freq': 700,
  'manufacturer': 'Commscope',
  'model': 'NNHH-65A-R4',
  'azimuth': 20,
  'agl': '60',
  'power': 141,
 

In [44]:
context = {
    'site_id': site_id,
    'address': address,
    'row_contents': row_contents
    }

In [45]:
doc = DocxTemplate('AntennaInventoryTpl.docx')
doc.render(context)
doc.save('test.docx')

In [46]:
dBm2W(W2dBm(661) - 12.4)

NameError: name 'WdBm' is not defined